# Using Hugging Face With Skills

In this notebook, we demonstrate using Hugging Face models for Skills using both SemanticMemory and text completions. 

SK supports downloading models from the Hugging Face that can perform the following tasks: text-generation, text2text-generation, summarization, and sentence-similarity. You can search for models by task at https://huggingface.co/models.

In [ ]:
!python -m pip install semantic-kernel==0.3.1.dev0

# Note that additional dependencies are required for the Hugging Face connectors:
!python -m pip install torch==2.0.0
!python -m pip install transformers==^4.28.1
!python -m pip install sentence-transformers==^2.2.2

In [ ]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.hugging_face as sk_hf

First, we will create a kernel and add both text completion and embedding services. 

For text completion, we are choosing GPT2. This is a text-generation model. (Note: text-generation will repeat the input in the output, text2text-generation will not.)
For embeddings, we are using sentence-transformers/all-MiniLM-L6-v2. Vectors generated for this model are of length 384 (compared to a length of 1536 from OpenAI ADA).

The following step may take a few minutes when run for the first time as the models will be downloaded to your local machine.

In [ ]:
kernel = sk.Kernel()

# Configure LLM service
kernel.add_text_completion_service(
    "gpt2", sk_hf.HuggingFaceTextCompletion("gpt2", task="text-generation")
)
kernel.add_text_embedding_generation_service(
    "sentence-transformers/all-MiniLM-L6-v2",
    sk_hf.HuggingFaceTextEmbedding("sentence-transformers/all-MiniLM-L6-v2"),
)
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

### Add Memories and Define a skill to use them

Most models available on huggingface.co are not as powerful as OpenAI GPT-3+. Your skills will likely need to be simpler to accommodate this.

In [ ]:
await kernel.memory.save_information_async(
    "animal-facts", id="info1", text="Sharks are fish."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info2", text="Whales are mammals."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info3", text="Penguins are birds."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info4", text="Dolphins are mammals."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info5", text="Flies are insects."
)

# Define semantic function using SK prompt template language
my_prompt = """I know these animal facts: {{recall $query1}} {{recall $query2}} {{recall $query3}} and """

# Create the semantic function
my_function = kernel.create_semantic_function(
    my_prompt, max_tokens=45, temperature=0.5, top_p=0.5
)


Let's now see what the completion looks like! Remember, "gpt2" is nowhere near as large as ChatGPT, so expect a much simpler answer.

In [ ]:
context = kernel.create_new_context()
context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = "animal-facts"
context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.3

context["query1"] = "animal that swims"
context["query2"] = "animal that flies"
context["query3"] = "penguins are?"
output = await kernel.run_async(my_function, input_vars=context.variables)

output = str(output).strip()


query_result1 = await kernel.memory.search_async(
    "animal-facts", context["query1"], limit=1, min_relevance_score=0.3
)
query_result2 = await kernel.memory.search_async(
    "animal-facts", context["query2"], limit=1, min_relevance_score=0.3
)
query_result3 = await kernel.memory.search_async(
    "animal-facts", context["query3"], limit=1, min_relevance_score=0.3
)

print(f"gpt2 completed prompt with: '{output}'")